In [15]:
from pymldb import Connection
mldb = Connection()

In [35]:
# The inception model retrained using food dataset with tensorflow 0.10.0
inceptionUrl = 'file://retrained_inception_new.zip'

In [18]:
# This sets up a fetcher function, which will download a given URL
# and returns it as a blob.
mldb.put('/v1/functions/fetch', {
    "id": 'fetch',
    "type": 'fetcher',
    "params": {
    }
})


<Response [201]>

In [25]:
# The labels for the retrained model live within the zip file
# above. We read them into a dataset so that we can
# join against them later on and turn category numbers into category
# names.

mldb.put('/v1/procedures/foodLabels', {
    "type": 'import.text',
    "params": {
        "dataFileUrl": 'archive+' + inceptionUrl + '#retrained_labels.txt',
        "headers": ['label'],
        "outputDataset": "foodLabels",
        "runOnCreation": True
    }
})

<Response [201]>

In [32]:
# This function takes the output of an inception (retrained) graph
# and joins the 11 scores against the image labels,
# producing a result set that contains an ordered set of category
# labels.  The line numbers of the dataset start at 1, so we need to
# subtract one to join with the label names.

mldb.put('/v1/functions/lookupLabels', {
    "type": 'sql.query',
    "params": {
        "query": """
            SELECT il.label AS column, scores.value AS value 
            FROM row_dataset($scores) AS scores 
            JOIN foodLabels AS il 
            ON CAST(scores.column AS INTEGER) = (CAST (il.rowName() AS INTEGER) - 1) 
            ORDER BY scores.value DESC 
            LIMIT 11
        """,
        "output": 'NAMED_COLUMNS'
    }
})

<Response [201]>

In [33]:
# Finally, we create the main function.  This is passed in a URL to
# classify as the url argument, and will download the image, process
# it through the inception net, and returns the top 11 categories with
# their weights as output.
#
# The image itself is fed into the DecodeJpeg/contents node, and the
# output is read from final_result node of the graph.

mldb.put('/v1/functions/imageEmbedding', {
    "type": 'tensorflow.graph',
    "params": {
        "modelFileUrl": 'archive+' + inceptionUrl + '#retrained_graph.pb',
        "inputs": 'fetch({url})[content] AS "DecodeJpeg/contents"',
        "outputs": "lookupLabels({scores: flatten(final_result)}) AS *"
    }
})

<Response [201]>

In [34]:
### Examples of images from the net

#filename = "http://butterybooks.com/wp-content/uploads/Biryani.jpg"  # Rice
#filename = "http://images.media-allrecipes.com/userphotos/560x315/1087142.jpg"   # Soup
filename = "http://assets.simplyrecipes.com/wp-content/uploads/2014/12/19140511/prime-rib-horiz-a-1600-600x400.jpg" # Meat

mldb.query("SELECT imageEmbedding({url: '%s'}) as *" % filename)

,output.bread,output.dairy product,output.dessert,output.egg,output.fried food,output.meat,output.noodles pasta,output.rice,output.seafood,output.soup,output.vegetables fruit
_rowName,,,,,,,,,,,
result,0.001744,0.001001,0.088869,0.000586,0.001101,0.90108,0.000126,0.000238,0.005069,0.000138,0.000048


In [14]:
#mldb.get('/v1/functions/imageEmbedding/details')